In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join('/kaggle/input/covid19-indonesia/kaggle datasets download -d brenda89/fifa-world-cup-2022'))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Data Wrangling
df = pd.read_csv('/kaggle/input/fifa-world-cup-2022/international_matches.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['date']=pd.to_datetime(df['date'])
df

In [ ]:
#Exploratory Data Analysis
fifa_rank = df[['date','home_team','away_team','home_team_fifa_rank','away_team_fifa_rank','away_team_total_fifa_points','home_team_total_fifa_points']]

In [ ]:
home = fifa_rank[['date','home_team','home_team_fifa_rank','home_team_total_fifa_points']].rename(columns = {'home_team':'team', 'home_team_fifa_rank':'rank','home_team_total_fifa_points':'rank_points'})
away = fifa_rank[['date','away_team','away_team_fifa_rank','away_team_total_fifa_points']].rename(columns = {'away_team':'team', 'away_team_fifa_rank':'rank','away_team_total_fifa_points':'rank_points'})
fifa_rank = home.append(away)
fifa_rank = fifa_rank.sort_values(['team','date'],ascending = [True,False])
fifa_rank['row_number']= fifa_rank.groupby('team').cumcount()+1
fifa_rank_top = fifa_rank[fifa_rank['row_number']==1].drop('row_number',axis=1).nsmallest(10,'rank')

#5 Strongest teams
fifa_rank_top

In [ ]:
#Checking Null Values
columns_contains_null = [col for col in df.columns if df[col].isnull().any()]
print(f'columns contain\'null\' values : \n\n (columns_contains_null)')

In [ ]:
wc_2022 = ['Qatar','Ecuador','Senegal','Netherland','England','Iran','USA','Wales','Argentina','Saudi Arabia','Mexico','Poland','France','Australia','Denmark','Tunisia','Spain','Costaa Rica','Germany','Japan','Belgium','Canada','Morocco','Croatia','Brazil','Serbia','Switzerland','Cameroon','Portugal','Ghana','Uruguay','South Korea']

In [ ]:
for x in wc_2022:
    for y in columns_contains_null:
        df[y].fillna(df[df[y[0:9]]== x][y].mean(), inplace = True)

In [ ]:
#Q1 - Is there such a thing as home team advantage?
home_team = df[['country','home_team_score','away_team_score']]
home_team.head()

In [ ]:
home_team['home_team_score']>home_team['away_team_score']
home_wins = home_team.loc[home_team['home_team_score']>home_team['away_team_score']]
home_wins

In [ ]:
home_team['home_team_score']<home_team['away_team_score']
home_loss = home_team.loc[home_team['home_team_score']<home_team['away_team_score']]
home_loss

In [ ]:
home_ad = home_wins[['country','home_team_score','away_team_score']].rename(columns = {'home_team_score' : 'home_team_win','away_team_score' : 'home_team_loss'})
away_ad = home_loss[['country','home_team_score','away_team_score']].rename(columns = {'home_team_score' : 'home_team_win','away_team_score' : 'home_team_loss'})

home_advantage = home_ad.append(away_ad)
home_advantage.head()

In [ ]:
home_advantage.mean().plot(kind = 'pie',title= 'Home Advantage vs Home Team Loss', labels = ['home_team_wins','home_team_loss'],figsize = (6,6))
plt.legend();

In [ ]:
#Q2 - Which team has the most wins?

hometeam = df[['date','home_team','home_team_score','away_team_score']]

hometeam.head(15)

In [ ]:
hometeam['home_team_score']>hometeam['away_team_score']
home_wins = hometeam.loc[home_team['home_team_score']>hometeam['away_team_score']]
home_wins.head()

In [ ]:
hometeam['home_team_score']<hometeam['away_team_score']
home_loss = hometeam.loc[home_team['home_team_score']<hometeam['away_team_score']]
home_loss.head()

In [ ]:
hometeam_wins = home_wins[['date','home_team','home_team_score','away_team_score']].rename(columns = {'home_team' : 'team','home_team_score' : 'home_team_win','away_team_score' : 'away_team_win'})
awayteam_wins = home_loss[['date','home_team','home_team_score','away_team_score']].rename(columns = {'home_team' : 'team','home_team_score' : 'home_team_win','away_team_score' : 'away_team_win'})
team_streak = hometeam_wins.append(awayteam_wins)
team_streak

In [ ]:
team_streak['team'].value_counts().nlargest(1)

In [ ]:
#Q3 - What is the winning percentage comparing when the highest-ranked team plays against the lowest-ranked team?

def victory(team1, team2, team1_rank, team2_rank, team1_score, team2_score):
    if team1_score > team2_score:
        winner = team1
    
    elif team1_score == team2_score:
        winner = 'Drawn'
    
    else:
        winner = team2
    
    return winner

In [ ]:
df['winner']= df .apply(lambda x: victory(x['home_team'], x['away_team'], x['home_team_fifa_rank'], x['away_team_fifa_rank']
                                          , x['home_team_score'], x['away_team_score']), axis=1)

In [ ]:
def rank(home,away,rank_home,rank_away):
    if rank_home < rank_away:
        return home
    else:
        return away

In [ ]:
df['better_rank'] = df.apply(lambda x: rank(x['home_team'], x['away_team'], 
                                            x['home_team_fifa_rank'],x['away_team_fifa_rank'])
                             , axis=1)

In [ ]:
def result_rank(winner,better_rank):
    if winner == 'Drawn':
        return winner
    if winner == better_rank:
        return 'Highest'
    else:
        return 'Smallest'

In [ ]:
df['result_rank']= df.apply(lambda x: result_rank(x['winner'], x['better_rank']), axis = 1)

In [ ]:
plt.figure(figsize = (10,5))
total = float(len(df))
ax = sns.countplot(x = 'result_rank', data=df)
for p in ax.patches :
    percentage = format(100*p.get_height()/total)
    
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.annotate(percentage, (x+width/2, y+height*1.02), ha='center')
    
plt.show()

In [ ]:
#Q4 - What is the winning percentage comparing when the highest attack rank plays against the lowest attack rank?

df_rank_attack = df[df['home_team_mean_offense_score'].notnull()]

In [ ]:
df_rank_attack = df[df['away_team_mean_offense_score'].notnull()]

In [ ]:
def rank_attack(home,away,rank_home,rank_away):
    if rank_home > rank_away:
        return home
    else:
        return away

In [ ]:
df.info()

In [ ]:
df_rank_attack['better_attack']= df.apply(lambda x: rank_attack(x['home_team'], x['away_team'], x['home_team_mean_offense_score']
                                                                , x['away_team_mean_offense_score']), axis= 1)

In [ ]:
df_rank_attack['result_attack'] = df_rank_attack.apply(lambda x: result_rank(x['winner'], x['better_attack']), axis =1)

In [ ]:
plt.figure(figsize = (10,5))
total = float(len(df_rank_attack))
ax = sns.countplot(x= 'result_attack', data = df_rank_attack)
for p in ax.patches:
    percentage = format(100*p.get_height()/total)
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.annotate(percentage, (x + width/2, y + height*1.02), ha = 'center')
plt.show()

In [ ]:
#Do teams with stronger offensive scores have more goals?

offense_score = []
for x in wc_2022:
    of_score = np.round((df[df['home_team']==x]['home_team_mean_offense_score'].mean()+df[df['away_team']==x]['away_team_mean_offense_score'].mean())/2, 2)
    offense_score.append(of_score)
offense_score = pd.DataFrame({'Team' : wc_2022, 'Of score': offense_score}).sort_values('Of score', ascending = False).reset_index(drop = True)
offense_score.index += 1
offense_score

In [ ]:
goals_conceded_home, goals_scored_home, goals_conceded_away, goals_scored_away = [], [], [], []
total_goals_scored, total_goals_conceded = [], []
for team in wc_2022:
    gch = df[df['home_team']==team]['away_team_score'].sum()
    gsh = df[df['home_team']==team]['home_team_score'].sum()
    gca = df[df['away_team']==team]['home_team_score'].sum()
    gsa = df[df['away_team']==team]['away_team_score'].sum()
    
    
    tgc = np.add(gch,gca)
    tgs = np.add(gsh,gsa)
    
    goals_conceded_home.append(gch)
    goals_scored_home.append(gsh)
    goals_conceded_away.append(gca)
    goals_scored_away.append(gsa)
    
    total_goals_scored.append(tgs)
    total_goals_conceded.append(tgc)
    
goals = pd.DataFrame({'Team':wc_2022,'Scored goals home':goals_scored_home,'Scored goals away':goals_scored_away,'Conceded goals home':goals_conceded_home,'Conceded goals away':goals_conceded_away ,'Total scored goals':total_goals_scored,'Total conceded goals':total_goals_conceded})

goals['Goal difference'] = goals['Total scored goals'] - goals['Total conceded goals']
goals['Goal ratio'] = goals['Total scored goals']/ goals['Total conceded goals']

goals = goals.sort_values('Goal difference', ascending = False).reset_index(drop = True)
goals.index += 1
goals

In [ ]:
plt.figure(figsize=(11,7), dpi=90)
ax = sns.barplot(data=goals[:10], x='Team', y='Goal difference')
plt.xlabel('TEAM')
plt.ylabel('GOAL DIFFERENCE')
plt.title('TOP 10 TEAMS BEST GOAL DIFFERENCE');

In [ ]:
#Top 10 Strongest Offensive Team

offense = df[['date', 'home_team', 'away_team', 'home_team_mean_offense_score', 'away_team_mean_offense_score']]
home = offense [['date', 'home_team','home_team_mean_offense_score']].rename(columns = {'home_team': 'team', 'home_team_mean_offense_score' : 'offense_score'})
away = offense [['date', 'away_team','away_team_mean_offense_score']].rename(columns = {'away_team': 'team', 'away_team_mean_offense_score' : 'offense_score'})
offense  = home.append(away)
# offense
#last match that each country played
offense   = offense .sort_values(['team', 'date'], ascending=[True, False])
offense ['row_number'] = offense .groupby('team').cumcount()+1
offense_top_data = offense [offense ['row_number']==1].drop('row_number',axis=1).nlargest(10, 'offense_score')
offense_top_data

In [ ]:
#Top 10 Strongest Defensive Teams?

defense = df[['date', 'home_team', 'away_team', 'home_team_mean_defense_score', 'away_team_mean_defense_score']]
home = defense [['date', 'home_team','home_team_mean_defense_score']].rename(columns = {'home_team': 'team', 'home_team_mean_defense_score' : 'defense_score'})
away = defense [['date', 'away_team','away_team_mean_defense_score']].rename(columns = {'away_team': 'team', 'away_team_mean_defense_score' : 'defense_score'})
defense  = home.append(away)
# offense
#last match that each country played
defense   = defense .sort_values(['team', 'date'], ascending=[True, False])
defense ['row_number'] = defense .groupby('team').cumcount()+1
defense_top_data = defense [offense ['row_number']==1].drop('row_number',axis=1).nlargest(10, 'defense_score')
defense_top_data

In [ ]:
#Top 10 Team that have Best Midfielder?

midfield = df[['date', 'home_team', 'away_team', 'home_team_mean_midfield_score', 'away_team_mean_midfield_score']]
home = midfield [['date', 'home_team','home_team_mean_midfield_score']].rename(columns = {'home_team': 'team', 'home_team_mean_midfield_score' : 'midfield_score'})
away = midfield [['date', 'away_team','away_team_mean_midfield_score']].rename(columns = {'away_team': 'team', 'away_team_mean_midfield_score' : 'midfield_score'})
midfield  = home.append(away)
# offense
#last match that each country played
midfield   = midfield .sort_values(['team', 'date'], ascending=[True, False])
midfield ['row_number'] = defense .groupby('team').cumcount()+1
midfield_top_data = midfield [offense ['row_number']==1].drop('row_number',axis=1).nlargest(10, 'midfield_score')
midfield_top_data

In [ ]:
#Top 10 Team which have strongest Defender, midfielder, offenser?

df['home_total_score'] = (df['home_team_mean_offense_score'] + df['home_team_mean_defense_score']+df['home_team_mean_midfield_score'])/3
df['away_total_score'] = (df['away_team_mean_offense_score'] + df['away_team_mean_defense_score']+df['away_team_mean_midfield_score'])/3
df['diff_total_score'] = df['home_total_score'] - df['away_total_score']

In [ ]:
total_score = df[['date', 'home_team', 'away_team', 'home_total_score', 'away_total_score']]
home = total_score [['date', 'home_team','home_total_score']].rename(columns = {'home_team': 'team', 'home_total_score' : 'total_score'})
away = total_score [['date', 'away_team','away_total_score']].rename(columns = {'away_team': 'team', 'away_total_score' : 'total_score'})
total_score  = home.append(away)
# offense
#last match that each country played
total_score   = total_score .sort_values(['team', 'date'], ascending=[True, False])
total_score ['row_number'] = defense .groupby('team').cumcount()+1
total_score_top_10 = total_score [offense ['row_number']==1].drop('row_number',axis=1).nlargest(10, 'total_score')
total_score_top_10

In [ ]:
#Machine Learning
#Simulation

# feature generation
df['rank_difference'] = df['home_team_fifa_rank'] - df['away_team_fifa_rank']
df['average_rank'] = (df['home_team_fifa_rank'] + df['away_team_fifa_rank'])/2
df['point_difference'] = df['home_team_total_fifa_points'] - df['away_team_total_fifa_points']
df['score_difference'] = df['home_team_score'] - df['away_team_score']
df['is_won'] = df['score_difference'] > 0 # take draw as lost
df['is_stake'] = df['tournament'] != 'Friendly'

In [ ]:
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

#X, y = df.loc[:,['average_rank', 'rank_difference', 'point_difference','is_stake']], df['is_won']
X, y = df.loc[:,['average_rank', 'rank_difference', 'point_difference']], df['is_won']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

logreg = linear_model.LogisticRegression(C=1e-5)
features = PolynomialFeatures(degree=2)
model = Pipeline([
    ('polynomial_features', features),
    ('logistic_regression', logreg)
])
model = model.fit(X_train, y_train)

# figures 
fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
plt.figure(figsize=(15,5))
ax = plt.subplot(1,3,1)
ax.plot([0, 1], [0, 1], 'k--')
ax.plot(fpr, tpr)
ax.set_title('AUC score is {0:0.2}'.format(roc_auc_score(y_test, model.predict_proba(X_test)[:,1])))
ax.set_aspect(1)

ax = plt.subplot(1,3,2)
cm = confusion_matrix(y_test, model.predict(X_test))
ax.imshow(cm, cmap='Blues', clim = (0, cm.max())) 

ax.set_xlabel('Predicted label')
ax.set_title('Performance on the Test set')

ax = plt.subplot(1,3,3)
cm = confusion_matrix(y_train, model.predict(X_train))
ax.imshow(cm, cmap='Blues', clim = (0, cm.max())) 
ax.set_xlabel('Predicted label')
ax.set_title('Performance on the Training set')
pass

In [ ]:
#List of all Teams in 2022 World Cup
teams_worldcup = ['Qatar', 'Ecuador', 'Senegal', 'Netherlands', 'England', 'Iran', 'USA',
                  'Wales', 'Argentina', 'Saudi Arabia', 'Mexico', 'Poland', 'France', 
                  'Australia', 'Denmark', 'Tunisia', 'Spain', 'Costa Rica', 'Germany', 
                  'Japan', 'Belgium', 'Canada', 'Morocco', 'Croatia', 'Brazil', 'Serbia', 
                  'Switzerland', 'Cameroon', 'Portugal', 'Ghana', 'Uruguay', 'South Korea']

In [ ]:
world_cup_rankings_home = df[['home_team','home_team_fifa_rank','home_team_total_fifa_points']].loc[df['home_team'].isin(teams_worldcup) & (df['date']>'2021-01-01')] #Get last 2 years of information (try to get all time too, get interesting results!)
world_cup_rankings_away = df[['away_team','away_team_fifa_rank','away_team_total_fifa_points']].loc[df['away_team'].isin(teams_worldcup)& (df['date']>'2021-01-01')]
world_cup_rankings_home = world_cup_rankings_home.set_index(['home_team'])

In [ ]:
#The idea is to separete the performance of each Team as Home or Away.
world_cup_rankings_home = world_cup_rankings_home.groupby('home_team').mean()
world_cup_rankings_away = world_cup_rankings_away.groupby('away_team').mean()

world_cup_rankings_away.head(13)

In [ ]:
world_cup_rankings_away.index

In [ ]:
from tqdm import tqdm #SHOW TASK BAR

simulation_winners = list()
simulation_results_winners = list()
simulation_results_round16 = list()
simulation_df_round16 = list()
simulation_results_quarterfinal = list()
simulation_df_quarterfinal = list()
simulation_results_semifinal = list()
simulation_df_semifinal = list()

n_simulations = 1000 #Number of Simulations

for j in tqdm(range(n_simulations)):
    #IMPORTANT: It has to sort the teams in the order of pairs they might go against starting from round_16 (i.e SENEGAL X NETHERLANDS, or FRANCE x DENMARK) - to play around with the matches check https://www.telegraph.co.uk/world-cup/2022-predictor-wallchart-football-qatar/
    candidates = [ 'Senegal', 'Netherlands', 'England','USA', 'Argentina', 'Poland', 'France', 'Denmark','Spain', 'Germany', 'Belgium', 'Croatia', 'Brazil', 'Serbia', 'Portugal','Uruguay']
    finals = ['round_of_16', 'quarterfinal', 'semifinal', 'final']
    
    for f in finals:
        iterations = int(len(candidates) / 2)
        #print('STEP:', iterations) #uncomment to observe in more details the iteraction
        winners = []
        prob = []


        for i in range(iterations):
            home = candidates[i*2]
            away = candidates[i*2+1]
            
            row = pd.DataFrame(np.array([[np.nan, np.nan, True]]), columns=X_test.columns)
            home_rank = world_cup_rankings_home.loc[home, 'home_team_fifa_rank']
            home_points = world_cup_rankings_home.loc[home, 'home_team_total_fifa_points']
            opp_rank = world_cup_rankings_away.loc[away, 'away_team_fifa_rank']
            opp_points = world_cup_rankings_away.loc[away, 'away_team_total_fifa_points']
            row['average_rank'] = (home_rank + opp_rank) / 2
            row['rank_difference'] = home_rank - opp_rank
            row['point_difference'] = home_points - opp_points
            home_win_prob = model.predict_proba(row)[:,1][0]
            
            # simulation step based on the probability
            simulated_outcome = np.random.binomial(1, home_win_prob)
            winners.append(away) if simulated_outcome <= 0.5 else winners.append(home)
            prob.append(1 - simulated_outcome) if simulated_outcome <= 0.5 else prob.append(simulated_outcome)
        
        if f == 'round_of_16':
            step_16 = ['round_16'] * 8
            candidates_round_16 = zip(step_16, winners, prob)
            df_candidates_round_16 = pd.DataFrame(candidates_round_16, columns = ['Step','Team','Prob'])
            simulation_df_round16.append(df_candidates_round_16)
            simulation_results_round16.append(winners)

        if f == 'quarterfinal':
            step_quarterfinal = ['quarterfinal'] * 4
            candidates_quarterfinal = zip(step_quarterfinal,winners, prob)
            df_candidates_quarterfinal = pd.DataFrame(candidates_quarterfinal, columns = ['Step','Team','Prob'])
            simulation_df_quarterfinal.append(df_candidates_quarterfinal)
            simulation_results_quarterfinal.append(winners)

        if f == 'semifinal':    
            step_semifinal = ['semifinal'] * 2
            candidates_semifinal = zip(step_semifinal,winners, prob)
            df_candidates_semifinal = pd.DataFrame(candidates_semifinal, columns = ['Step','Team','Prob'])
            simulation_df_semifinal.append(df_candidates_semifinal)
            simulation_results_semifinal.append(winners)

        if f == 'final':    
            step_final = ['final'] * 1
            candidates_final = zip(step_final,winners, prob)
            df_candidates_final = pd.DataFrame(candidates_final, columns = ['Step','Team','Prob'])
            simulation_winners.append(df_candidates_final)
            simulation_results_winners.append(winners)

        
        #uncomment to observe every result of each step and iteration
        #list_win = list(zip(winners,prob))
        #print(list_win)
        candidates = winners 

df_candidates_round_16 = pd.concat(simulation_df_round16)
df_candidates_quarterfinal = pd.concat(simulation_df_quarterfinal)
df_candidates_semifinal = pd.concat(simulation_df_semifinal)
df_candidates_final = pd.concat(simulation_winners)

df_results = pd.concat([df_candidates_round_16,df_candidates_quarterfinal,df_candidates_semifinal,df_candidates_final]) #final DataFrame

simulation_results_round16 = sum(simulation_results_round16, [])
simulation_results_quarterfinal = sum(simulation_results_quarterfinal, [])
simulation_results_semifinal = sum(simulation_results_semifinal, [])
simulations_winners = sum(simulation_results_winners, [])

In [ ]:
#Counter is used to easily count recurrent elements in a list check this link for more info (https://realpython.com/python-counter/)
from collections import Counter

In [ ]:
from matplotlib import figure

lst_results = [simulation_results_round16,simulation_results_quarterfinal,simulation_results_semifinal,simulations_winners]
for i in lst_results:
    results = Counter(i).most_common()
    x,y = zip(*results)

    fig,ax = plt.subplots(figsize=(16,8))

    ax.barh(x, y)

    ax.set_ylabel('Team')

    if i == simulation_results_round16:
        ax.set_title('Round6 World Cup 2022')
    if i == simulation_results_quarterfinal:
        ax.set_title('QuarterFinal World Cup 2022')
    if i == simulation_results_semifinal:
        ax.set_title('SemiFinal World Cup 2022')
    if i == simulations_winners:
        ax.set_title('Winner World Cup 2022')

        
    plt.show()

In [ ]:
#Join all data in onw single view
import seaborn as sns

sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(14,10)})

# Draw a nested barplot by Team and Step
g = sns.barplot(data=df_results, x="Team",y='Prob', hue="Step", estimator=sum, palette="dark", alpha=.6)

In [ ]:
#Winner Prediction of World Cup 2022 is Brazil

Interpretation :
1. France has most win-rate in round-16 until semi-final World Cup 2022
2. Senegal and Serbia most likely the weakest teams, for less appearence in Finals;
3. Top 3 Teams are Brazil, Belgium and France with more than 10% chances of winning the tournment.
4. Team which has most win chance to win the world cup 2022 is Brazil